In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

In [3]:
train_df.head(3)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}


In [4]:
train_df.describe()

,id,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,ogrn_days_end_month,ogrn_days_end_quarter,ft_registration_date,max_founderpres,min_founderpres,...,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m
count,600000.000000,5.344230e+05,5.344230e+05,5.344230e+05,5.344230e+05,5.685270e+05,5.685270e+05,5.686400e+05,2.644080e+05,2.644080e+05,...,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000
mean,99999.500000,5.291614e-18,-7.312532e-17,-3.959403e-17,-9.945043e-18,-6.117751e-17,1.003586e-17,-2.430370e-18,1.211971e-17,-3.759528e-17,...,0.810852,7.401487e-19,0.946172,0.410920,3.422448e-18,0.551292,0.507565,4.041212e-17,0.255424,0.456226
std,57735.075031,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000002e+00,1.000002e+00,...,0.145072,1.000001e+00,0.011519,0.080259,1.000001e+00,0.014255,0.156156,1.000001e+00,0.017320,0.136101
min,0.000000,-1.567780e-01,-2.049624e-01,-1.259950e-01,-1.563145e-01,-1.649833e+00,-1.722458e+00,-1.081749e+00,-1.149405e+00,-1.075208e+00,...,0.728507,-2.858428e-02,0.944497,0.384773,-1.655883e-01,0.546889,0.407687,-2.011323e-01,0.250924,0.374540
25%,49999.750000,-1.560927e-01,-2.042761e-01,-1.259122e-01,-1.556526e-01,-8.369370e-01,-8.706851e-01,-7.562179e-01,-7.608814e-01,-7.668686e-01,...,0.728507,-2.858399e-02,0.944497,0.384773,-1.655881e-01,0.546889,0.407687,-2.011227e-01,0.250924,0.374540
50%,99999.500000,-1.499485e-01,-1.903097e-01,-1.248704e-01,-1.495861e-01,-2.404093e-02,-1.891231e-02,-3.151202e-01,-2.764884e-01,-2.965527e-01,...,0.750485,-2.858399e-02,0.944497,0.384773,-1.655881e-01,0.547176,0.429909,-2.011227e-01,0.251501,0.396518
75%,149999.250000,-1.033781e-01,-1.063255e-01,-1.124800e-01,-1.033203e-01,7.888552e-01,8.715774e-01,4.040237e-01,4.519933e-01,4.554324e-01,...,0.816419,-2.858399e-02,0.944889,0.396267,-1.436181e-01,0.549755,0.541020,-1.384441e-01,0.254101,0.473441
max,199999.000000,1.542407e+02,1.174957e+02,9.145077e+01,1.547193e+02,1.834007e+00,1.800784e+00,3.032875e+00,3.376642e+00,3.592823e+00,...,1.728507,3.659603e+02,1.944497,1.384773,2.474600e+02,1.546889,1.407687,1.075818e+02,1.250924,1.374540


In [5]:
train_df.shape

(600000, 93)

In [6]:
train_df["start_cluster"][:20]

0      {α, γ}
1      {α, γ}
2      {α, γ}
3     {other}
4     {other}
5     {other}
6         {α}
7         {α}
8         {α}
9         {α}
10        {α}
11        {α}
12        {α}
13        {α}
14        {α}
15         {}
16         {}
17        {α}
18     {α, γ}
19     {α, γ}
Name: start_cluster, dtype: object